# GRASS Tool in C

For some tools, C (or C++) is a good choice of language because an algorithm or model requires fine work with the data. For example, a loop in Python over all vector points or raster cells may be prohibitively slow. When this cannot be compensated by available Python libraries or use of GRASS C libraries through the ctypes API or grass.pygrass wrappers, writing a tool directly in C is the most common approach.

GRASS tools written in C (and C++) can be distributed in exactly the same way as tools in Python. For Windows, they are compiled on project servers. For other systems, they are compiled on the user machine. Python, C, and C++ tools can be and are being published in the grass-addons repository with users noticing the difference only in case of bugs.

## Structure of a C Tool

There is no difference for small tools comparing to Python tools except that the file with the source code is called main. There are still three files: source code, documentation, and a Makefile.

Explore the directory with example tool called _r.example.twice_ which multiplies values in a raster map by two:

In [ ]:
!ls r.example.twice/

Open _r.example.twice/main.c_ (using File Browser in JupyterLab).

The file with the _main_ function should be called `main.c`. There can be multiple `.c` and `.h` files.

## Compile and Install

Here, we will compile and install the tool in the same way as a Python tool using _g.extension_: 

In [ ]:
!grass --tmp-location XY --exec g.extension r.example.twice url=r.example.twice

In case of C, compilation is necessary to execute the program, so typically, the low-level command _make_ is used to do the compilation. The specific use may differ based on the development environment, but often you may want to compile whole GRASS GIS yourself and simply add the compiled tool to that build instead of installing it as an addon (which is what happens with _g.extension_). In that case, _make_ is used with `MODULE_TOPDIR` variable which is set to where the GRASS source code is, e.g.:

```bash
make MODULE_TOPDIR=~/Projects/grass/
```

If you are calling `make install` to install GRASS GIS after compilation, you need to do the same with your code:

```bash
make install MODULE_TOPDIR=~/Projects/grass/
```

This assumes you are developing a tool for grass-addons. If you are developing a tool for the main repository, `MODULE_TOPDIR=...` is not needed.

Although this is most useful for developing tools in C, the same applies for Python tools as well.

Using _g.extension_ with the `-d` flag, gives suggestion of a setup for complex cases in case _g.extension_ or the usage of _make_ above are not enough.

## Test of the Interface

The resulting interface is the same as for Python tools, for example `--help` works:

In [ ]:
!grass --tmp-location XY --exec r.example.twice --help

## Test of the Computation

Let's test the tool with raster _elevation_. First, check its metadata and then set the computational region to it:

In [ ]:
!grass ~/grassdata/nc_basic_spm_grass7/foss4g --exec r.info map=elevation
!grass ~/grassdata/nc_basic_spm_grass7/foss4g --exec g.region raster=elevation

Now run the _r.example.twice_ tool (we use `--o` in the example as shorthand for `--overwrite`, so we can re-run the example multiple times):

In [ ]:
!grass ~/grassdata/nc_basic_spm_grass7/foss4g --exec r.example.twice input=elevation output=test_twice --o

Minimum and maximum in metadata of the new raster should be double the original value:

In [ ]:
!grass ~/grassdata/nc_basic_spm_grass7/foss4g --exec r.info map=test_twice

We will use a subprocess to render the result as a PNG image (without modifying our current environment):

In [ ]:
%%python
import subprocess
import sys

sys.path.append(
    subprocess.check_output(["grass", "--config", "python_path"], text=True).strip()
)

import grass.script as gs
import grass.jupyter as gj
import grass.script.setup  # Needed only in 8.2 and older.

with grass.script.setup.init("~/grassdata/nc_basic_spm_grass7/foss4g") as session:
    ortho_map = gj.Map()
    ortho_map.d_rast(map="test_twice")
    # Save the image (in a standard notebook, we would just display the image now).
    ortho_map.save("test_twice.png")

Display the image in the notebook:

In [ ]:
from IPython.display import Image

Image("test_twice.png")